In [3]:
# encoding:utf-8
import numpy as np
import pandas as pd
import pandas as pd
# import matplotlib.pyplot as plt
from fbprophet import Prophet
import pystan

d:\Anaconda\envs\prophet\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


处理步骤：

1. 新增确诊以及死亡数据作为元数据

2. 预测新增确诊与死亡

3. 将预测结果与原始数据相连接

4. 生成累计确诊与死亡数据

5. 处理成js读取的json格式

1. 新增确诊以及死亡数据作为元数据

2. 预测未来新增与死亡

In [34]:
from predict import predict

### 数据导入
# input_data = pd.read_csv(input_path)
# pdata.to_csv(save_path,index=False)
def pd_read_csv(path):
    """读取文件
    """
    data = pd.read_csv(path)
    return data

def generate_future(d_confirm,d_death,pred_day=90):
    """时间: 30min, 生成预测数据，并保存到相应文件夹
    :params d_confirm d_death 新增确诊与死亡数据
    :return p_confirm p_death 预测死亡与确诊
    """
    p_confirm = predict(d_confirm,pred_day)
    p_confirm.to_csv("./data/predict_confirm.csv",index=False)

    p_death = predict(d_death,pred_day)
    p_death.to_csv("./data/predict_death.csv",index=False)
    return p_confirm,p_death

# 3. 将预测结果与原始数据相连接
def concat_predict(d_confirm,d_death,p_confirm,p_death):
    """将预测数据与原始数据连接一起
    :params 输入新增与预测的（确诊与死亡）
    :return 连接后的新增与死亡
    """
    # load the prediction
    p_confirm = pd.read_csv('./data/predict_confirm.csv')
    p_death = pd.read_csv('./data/predict_death.csv')

    # change datetime format
    p_confirm['Date'] = pd.to_datetime(p_confirm['Date']).dt.strftime('%Y/%m/%d')
    p_death['Date'] = pd.to_datetime(p_death['Date']).dt.strftime('%Y/%m/%d')

    # get the result
    a_confirm = pd.concat([d_confirm, p_confirm],axis=0).reset_index(drop=True)
    a_death = pd.concat([d_death, p_death],axis=0).reset_index(drop=True)

    return a_confirm, a_death

# 4. 生成累计确诊与死亡数据
def generate_cumulative(daily_pd):
    """
    :params daily_pd
    :return cumulative_pd
    """
    cum_pd = pd.DataFrame()
    without_date = daily_pd.columns[daily_pd.columns!='Date']
    cum_pd['Date'] = daily_pd['Date']
    for column in without_date:
        cum_pd[column] = daily_pd[column].cumsum()
    return cum_pd

# 5. 处理成js读取的json格式
def generate_json(a_confirm,a_death,save_path = 'plot.json'):
    """a_confirm,a_death,save_path = 'plot.json'
    :params a_confirm a_death 新增确诊与死亡（包括预测部分）
    :params save_path 最终json文件的保存位置 
    :return None
    """
    # 生成累计数据
    c_confirm = generate_cumulative(a_confirm)
    c_death = generate_cumulative(a_death)

    result = dict()
    without_date = a_confirm.columns[a_confirm.columns!='Date']
    for column in without_date:
        temp = pd.DataFrame()
        temp['Date'] = a_confirm['Date']
        temp[column+'1'] = a_confirm[column]
        temp[column+'2'] = c_confirm[column]
        temp[column+'3'] = a_death[column]
        temp[column+'4'] = c_death[column]
        temp = temp.values.tolist()
        result[column] = temp
    import json
    with open(save_path,"w") as f:
        json.dump(result,f)

In [1]:
# 默认有预测数据，从第三步开始
d_confirm_path = './data/daily_confirm.csv'
d_death_path = './data/daily_death.csv'
p_confirm_path = './data/predict_confirm.csv'
p_death_path = './data/predict_death.csv'
save_path = './data/plot.json'

from update import Update
update = Update()
update.update_json_without_predict(90)

d:\Anaconda\envs\prophet\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Beginning predict: --------------------------------


  0%|          | 0/34 [00:00<?, ?it/s]e:\django\infectious_disease_analysis_platform\visual\core_function\predict.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['yhat'][test['yhat']<0] = 0 #  负数改为0
  3%|▎         | 1/34 [00:08<04:30,  8.20s/it]e:\django\infectious_disease_analysis_platform\visual\core_function\predict.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['yhat'][test['yhat']<0] = 0 #  负数改为0
  6%|▌         | 2/34 [00:15<03:56,  7.39s/it]e:\django\infectious_disease_analysis_platform\visual\core_function\predict.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

KeyboardInterrupt: 

In [21]:
import pandas as pd

a = pd.DataFrame(
    data=['2012-08-29',
    '2012-12-31',
    '2012-12-31'],columns=['Date']
)


In [22]:
from datetime import datetime
def date_format(date):
    year,month,day = date.split('-')
    return str(int (year)) + '/' + str(int (month)) + '/' + str(int(day))
a['Date'].apply(date_format)

0     2012/8/29
1    2012/12/31
2    2012/12/31
Name: Date, dtype: object